In [54]:
import numpy as np
import math # P2
import struct # P3
from typing import Generator # P3


In [55]:
# Problem 1: Binary Words and Operations

def Parity(x, y, z):
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    return x ^ y ^ z

def Ch(x, y, z):
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    return (x & y) ^ (~x & z)

def Maj(x, y, z):
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    return (x & y) ^ (x & z) ^ (y & z)

def Sigma0(x):
    x = np.uint32(x)
    # Inline rotation implementation
    rotr_2 = (x >> 2) | (x << (32 - 2))
    rotr_13 = (x >> 13) | (x << (32 - 13))
    rotr_22 = (x >> 22) | (x << (32 - 22))
    return rotr_2 ^ rotr_13 ^ rotr_22

def Sigma1(x):
    x = np.uint32(x)
    # Inline rotation implementation
    rotr_6 = (x >> 6) | (x << (32 - 6))
    rotr_11 = (x >> 11) | (x << (32 - 11))
    rotr_25 = (x >> 25) | (x << (32 - 25))
    return rotr_6 ^ rotr_11 ^ rotr_25

def sigma0(x):
    x = np.uint32(x)
    # Inline implementation
    rotr_7 = (x >> 7) | (x << (32 - 7))
    rotr_18 = (x >> 18) | (x << (32 - 18))
    shr_3 = x >> 3
    return rotr_7 ^ rotr_18 ^ shr_3

def sigma1(x):
    x = np.uint32(x)
    # Inline implementation
    rotr_17 = (x >> 17) | (x << (32 - 17))
    rotr_19 = (x >> 19) | (x << (32 - 19))
    shr_10 = x >> 10
    return rotr_17 ^ rotr_19 ^ shr_10

"Testing 7 required functions**************************************************"

# Test the 7 required functions
print(f"Parity(0x00000001, 0x00000001, 0x00000000) = {Parity(0x00000001, 0x00000001, 0x00000000):08x}")
print(f"Ch(0xFFFFFFFF, 0x12345678, 0x87654321) = {Ch(0xFFFFFFFF, 0x12345678, 0x87654321):08x}")
print(f"Maj(0xF0F0F0F0, 0xFF00FF00, 0x0F0F0F0F) = {Maj(0xF0F0F0F0, 0xFF00FF00, 0x0F0F0F0F):08x}")
print()

test_val = 0x12345678
print(f"Sigma0(0x12345678) = {Sigma0(test_val):08x}")
print(f"Sigma1(0x12345678) = {Sigma1(test_val):08x}")
print(f"sigma0(0x12345678) = {sigma0(test_val):08x}")
print(f"sigma1(0x12345678) = {sigma1(test_val):08x}")

Parity(0x00000001, 0x00000001, 0x00000000) = 00000000
Ch(0xFFFFFFFF, 0x12345678, 0x87654321) = 12345678
Maj(0xF0F0F0F0, 0xFF00FF00, 0x0F0F0F0F) = ff00ff00

Sigma0(0x12345678) = 66146474
Sigma1(0x12345678) = 3561abda
sigma0(0x12345678) = e7fce6ee
sigma1(0x12345678) = a1f78649


In [56]:
# Problem 2: SHA-256 Constants Generation

def primes(n):
    """
    Generate the first n prime numbers 
    """
    if not isinstance(n, int) or n <= 0:
        raise ValueError("n must be a positive integer")
    
    if n == 1:
        return [2]
    
    if n < 6:
        upper_bound = 20
    else:
        upper_bound = int(n * (math.log(n) + math.log(math.log(n)))) + 10
    
    sieve = [True] * (upper_bound + 1)
    sieve[0:2] = [False, False]
    
    primes_list = []
    for num in range(2, upper_bound + 1):
        if sieve[num]:
            primes_list.append(num)
            if len(primes_list) == n:
                break
            for multiple in range(num * num, upper_bound + 1, num):
                sieve[multiple] = False
    
    return primes_list

print("=== Testing Prime Number Generator ===\n")

test_cases = [1, 5, 10, 64]
for n in test_cases:
    prime_list = primes(n)
    print(f"First {n} primes: {prime_list}")
    print(f"Length: {len(prime_list)}, Last prime: {prime_list[-1]}\n")



"*********************************************************************************************************"



def get_fractional_bits_corrected(number, num_bits=32):
    """
    Extract the first num_bits of fractional part of the given number.
    """
    fractional = number - int(number)
    scaled = fractional * (2 ** num_bits)
    result = int(scaled)

    return result & ((1 << num_bits) - 1)

# Test fractional part extraction
print("=== Testing Fractional Part Bit Extraction ===\n")

test_primes = [2, 3, 5, 7]
for prime in test_primes:
    cube_root = prime ** (1.0/3.0)
    fractional_bits = get_fractional_bits_corrected(cube_root, 32)
    print(f"Prime: {prime}")
    print(f"Cube root: {cube_root:.10f}")
    print(f"Fractional part bits: {fractional_bits:08x}")
    print(f"Integer part: {int(cube_root)}, Fractional: {cube_root - int(cube_root):.10f}\n")



"*********************************************************************************************************"



def calculate_sha256_constants_corrected():
    """
    Calculate SHA-256 constants with exact FIPS PUB 180-4 specification.
    """
    prime_numbers = primes(64)
    constants = []
    
    for prime in prime_numbers:
        cube_root = prime ** (1.0/3.0)
        fractional_bits = get_fractional_bits_corrected(cube_root, 32)
        hex_constant = f"{fractional_bits:08x}"
        constants.append(hex_constant)
    
    return constants

# Calculate all SHA-256 constants
print("=== SHA-256 Constants Calculation ===\n")

sha256_constants = calculate_sha256_constants_corrected()

print("Generated SHA-256 Constants:")
print("-" * 40)
print(sha256_constants)
print(f"\nTotal constants generated: {len(sha256_constants)}")




"*********************************************************************************************************"



expected_constants = [
    '428a2f98', '71374491', 'b5c0fbcf', 'e9b5dba5', '3956c25b', '59f111f1', '923f82a4', 'ab1c5ed5',
    'd807aa98', '12835b01', '243185be', '550c7dc3', '72be5d74', '80deb1fe', '9bdc06a7', 'c19bf174',
    'e49b69c1', 'efbe4786', '0fc19dc6', '240ca1cc', '2de92c6f', '4a7484aa', '5cb0a9dc', '76f988da',
    '983e5152', 'a831c66d', 'b00327c8', 'bf597fc7', 'c6e00bf3', 'd5a79147', '06ca6351', '14292967',
    '27b70a85', '2e1b2138', '4d2c6dfc', '53380d13', '650a7354', '766a0abb', '81c2c92e', '92722c85',
    'a2bfe8a1', 'a81a664b', 'c24b8b70', 'c76c51a3', 'd192e819', 'd6990624', 'f40e3585', '106aa070',
    '19a4c116', '1e376c08', '2748774c', '34b0bcb5', '391c0cb3', '4ed8aa4a', '5b9cca4f', '682e6ff3',
    '748f82ee', '78a5636f', '84c87814', '8cc70208', '90befffa', 'a4506ceb', 'bef9a3f7', 'c67178f2'
]

print("=== Verification Against FIPS PUB 180-4 ===\n")

print("Generated Constants vs Expected:")
print("-" * 50)

matches = 0
for i in range(64):
    gen = sha256_constants[i]
    exp = expected_constants[i]
    status = "MATCH" if gen == exp else "MISMATCH"
    if gen == exp:
        matches += 1
    print(f"{i:2d}. {gen} {status:8} {exp}")
    

=== Testing Prime Number Generator ===

First 1 primes: [2]
Length: 1, Last prime: 2

First 5 primes: [2, 3, 5, 7, 11]
Length: 5, Last prime: 11

First 10 primes: [2, 3, 5, 7, 11, 13, 17, 19, 23, 29]
Length: 10, Last prime: 29

First 64 primes: [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311]
Length: 64, Last prime: 311

=== Testing Fractional Part Bit Extraction ===

Prime: 2
Cube root: 1.2599210499
Fractional part bits: 428a2f98
Integer part: 1, Fractional: 0.2599210499

Prime: 3
Cube root: 1.4422495703
Fractional part bits: 71374491
Integer part: 1, Fractional: 0.4422495703

Prime: 5
Cube root: 1.7099759467
Fractional part bits: b5c0fbcf
Integer part: 1, Fractional: 0.7099759467

Prime: 7
Cube root: 1.9129311828
Fractional part bits: e9b5dba5
I

In [ ]:
# Problem 3: Message Block Parsing
def block_parse(msg: bytes) -> Generator[bytes, None, None]:
    # Message length in bits
    msg_len_bits = len(msg) * 8
    
    padded = bytearray(msg)
    
    # append 1 bit as 0x80 byte
    padded.append(0x80)
    
    # calculate and append 0 bits
    current_bits = len(padded) * 8
    zero_bits = (448 - current_bits) % 512
    zero_bytes = (zero_bits + 7) // 8  # Ceiling division
    padded.extend(b'\x00' * zero_bytes)
    
    # append 64-bit message length
    padded.extend(struct.pack('>Q', msg_len_bits))
    
    # yield 512-bit blocks
    for i in range(0, len(padded), 64):
        yield bytes(padded[i:i+64])


"*********************************************************************************************************"

# Testing the block_parse Generator

# Test 1: Empty message
print("Test 1: Empty message (0 bytes)")
blocks = list(block_parse(b''))
print(f"  Blocks: {len(blocks)}")
print(f"  First byte: 0x{blocks[0][0]:02x} (0x80 = '1' bit)")
print(f"  Last 8 bytes: {blocks[0][-8:].hex()} (length = 0)")
print()

# Test 2: "abc"
print("Test 2: 'abc' (3 bytes, FIPS standard example)")
blocks = list(block_parse(b'abc'))
print(f"  Blocks: {len(blocks)}")
block_hex = blocks[0].hex()
print(f"  Block starts: {block_hex[:32]}...")
print(f"  Block ends: ...{block_hex[-16:]}")
print(f"  Length field: {struct.unpack('>Q', blocks[0][-8:])[0]} bits")
print()

# Test 3: Exactly 55 bytes
print("Test 3: Exactly 55 bytes")
msg = b'A' * 55
blocks = list(block_parse(msg))
combined = b''.join(blocks)
print(f"  Blocks: {len(blocks)}")
# Find 0x80 in the combined padded message
pos = combined.find(b'\x80')
print(f"  0x80 position: byte {pos}")
if pos == 55:
    print(f"  Correct: 0x80 at position 55")
else:
    print(f"  ERROR: 0x80 at wrong position {pos}")
print()

# Test 4: Exactly 56 bytes
print("Test 4: Exactly 56 bytes")
msg = b'B' * 56
blocks = list(block_parse(msg))
combined = b''.join(blocks)
print(f"  Blocks: {len(blocks)}")
print(f"  Block 1 has message: {blocks[0][:56].hex()[:16]}...")
# Find 0x80 in the combined padded message
pos = combined.find(b'\x80')  
print(f"  0x80 position: byte {pos}")
if pos == 56:
    print(f"  Correct: 0x80 at position 56")
else:
    print(f"  ERROR: 0x80 at wrong position {pos}")
print()

# Test 5: 100 bytes
print("Test 5: Long message")
msg = b'C' * 100
blocks = list(block_parse(msg))
print(f"  Blocks: {len(blocks)}")
print(f"  All blocks 64 bytes: {all(len(b) == 64 for b in blocks)}")
print(f"  Total padded length: {len(blocks) * 64} bytes")
print()


# Demonstration of Generator Behavior

"""
48656c6c6f2c2057 = Hex for "Hello, W" 

48 = H, 65 = e, 6c = l, 6c = l, 6f = o, 2c = ,, 20 = space, 57 = W

0000000000000068 = Length in bits (104 bits = 13 bytes × 8)
"""

print("'Hello, World!':")
gen = block_parse(b'Hello, World!')

for i, block in enumerate(gen):
    print(f"  Yielded block {i+1}: {len(block)} bytes")
    print(f"    First 8 bytes: {block[:8].hex()}")
    print(f"    Last 8 bytes: {block[-8:].hex()}")
    print()

Test 1: Empty message (0 bytes)
  Blocks: 1
  First byte: 0x80 (0x80 = '1' bit)
  Last 8 bytes: 0000000000000000 (length = 0)

Test 2: 'abc' (3 bytes, FIPS standard example)
  Blocks: 1
  Block starts: 61626380000000000000000000000000...
  Block ends: ...0000000000000018
  Length field: 24 bits

Test 3: Exactly 55 bytes
  Blocks: 1
  0x80 position: byte 55
  Correct: 0x80 at position 55

Test 4: Exactly 56 bytes
  Blocks: 2
  Block 1 has message: 4242424242424242...
  0x80 position: byte 56
  Correct: 0x80 at position 56

Test 5: Long message
  Blocks: 2
  All blocks 64 bytes: True
  Total padded length: 128 bytes

'Hello, World!':
  Yielded block 1: 64 bytes
    First 8 bytes: 48656c6c6f2c2057
    Last 8 bytes: 0000000000000068



In [58]:
# Fix Problem 2
def primes(n):
    if not isinstance(n, int) or n <= 0:
        raise ValueError("n must be a positive integer")
    
    if n == 1:
        return [2]
    
    # Estimate upper bound for nth prime
    if n < 6:
        upper_bound = 20  # Small n heuristic
    else:
        upper_bound = int(n * (np.log(n) + np.log(np.log(n)))) + 10
    
    # Boolean array for sieve - True indicates prime candidate
    sieve = np.ones(upper_bound + 1, dtype=bool)
    sieve[0:2] = False 
    
    primes_found = []
    for current in range(2, upper_bound + 1):
        if sieve[current]:
            primes_found.append(current)
            if len(primes_found) == n:
                break
            sieve[current*current:upper_bound+1:current] = False
    
    return primes_found

test_counts = [1, 5, 10, 64]
for count in test_counts:
    prime_sequence = primes(count)
    print(f"First {count} primes: {prime_sequence}")
    print(f"Count: {len(prime_sequence)}, Largest: {prime_sequence[-1]}")
    
    # Verify we can convert to 32-bit numpy array for SHA-256
    if count == 64:
        prime_array = np.array(prime_sequence, dtype=np.uint32)
        print(f"Verification: Array shape={prime_array.shape}, Type={prime_array.dtype}")
    
    print()

First 1 primes: [2]
Count: 1, Largest: 2

First 5 primes: [2, 3, 5, 7, 11]
Count: 5, Largest: 11

First 10 primes: [2, 3, 5, 7, 11, 13, 17, 19, 23, 29]
Count: 10, Largest: 29

First 64 primes: [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311]
Count: 64, Largest: 311
Verification: Array shape=(64,), Type=uint32



In [59]:
# Fix Problem 2
def extract_fractional_bits(value, num_bits=32):

    fractional, _ = np.modf(np.float64(value))

    scaled = fractional * np.float64(2 ** num_bits)
    
    result = np.uint32(scaled)
    
    # Ensure we only have the requested number of bits
    bitmask = (1 << num_bits) - 1
    return result & bitmask

# Test with first four primes as specified in FIPS 180-4
test_primes = [2, 3, 5, 7]
for prime in test_primes:
    cube_root = np.power(prime, 1.0/3.0, dtype=np.float64)
    
    fractional_bits = extract_fractional_bits(cube_root, 32)
    
    print(f"Prime {prime}:")
    print(f"  Cube root: {cube_root:.12f}")
    print(f"  Fractional bits (hex): 0x{fractional_bits:08x}")
    print(f"  Fractional bits (dec): {fractional_bits}")
    print()

Prime 2:
  Cube root: 1.259921049895
  Fractional bits (hex): 0x428a2f98
  Fractional bits (dec): 1116352408

Prime 3:
  Cube root: 1.442249570307
  Fractional bits (hex): 0x71374491
  Fractional bits (dec): 1899447441

Prime 5:
  Cube root: 1.709975946677
  Fractional bits (hex): 0xb5c0fbcf
  Fractional bits (dec): 3049323471

Prime 7:
  Cube root: 1.912931182772
  Fractional bits (hex): 0xe9b5dba5
  Fractional bits (dec): 3921009573



In [60]:
# Fix Problem 2
def compute_sha256_constants():
    prime_sequence = primes(64)
    
    hex_constants = []
    constant_values = []
    
    for prime in prime_sequence:
        cube_root = np.power(prime, 1.0/3.0, dtype=np.float64)
        
        constant = extract_fractional_bits(cube_root, 32)
        
        hex_constants.append(f"{constant:08x}")
        constant_values.append(constant)
    
    K256 = np.array(constant_values, dtype=np.uint32)
    
    return hex_constants, K256

hex_constants, K256 = compute_sha256_constants()

print(f"  Elements: {K256.shape[0]}")
print(f"  Data Type: {K256.dtype} (32-bit unsigned integer)")
print(f"  Memory: {K256.nbytes} bytes")
print(f"  First: 0x{K256[0]:08x}, Last: 0x{K256[-1]:08x}")
print()

print("Constants in Standard SHA-256 Format:")
print("-" * 40)

def format_hex_constants(constants, per_line=8):
    """Format constants in standard 8-column layout."""
    return "\n".join(
        " ".join(constants[i:i+per_line])
        for i in range(0, len(constants), per_line)
    )

print(format_hex_constants(hex_constants))
print(f"\nTotal: {len(hex_constants)} constants")

  Elements: 64
  Data Type: uint32 (32-bit unsigned integer)
  Memory: 256 bytes
  First: 0x428a2f98, Last: 0xc67178f2

Constants in Standard SHA-256 Format:
----------------------------------------
428a2f98 71374491 b5c0fbcf e9b5dba5 3956c25b 59f111f1 923f82a4 ab1c5ed5
d807aa98 12835b01 243185be 550c7dc3 72be5d74 80deb1fe 9bdc06a7 c19bf174
e49b69c1 efbe4786 0fc19dc6 240ca1cc 2de92c6f 4a7484aa 5cb0a9dc 76f988da
983e5152 a831c66d b00327c8 bf597fc7 c6e00bf3 d5a79147 06ca6351 14292967
27b70a85 2e1b2138 4d2c6dfc 53380d13 650a7354 766a0abb 81c2c92e 92722c85
a2bfe8a1 a81a664b c24b8b70 c76c51a3 d192e819 d6990624 f40e3585 106aa070
19a4c116 1e376c08 2748774c 34b0bcb5 391c0cb3 4ed8aa4a 5b9cca4f 682e6ff3
748f82ee 78a5636f 84c87814 8cc70208 90befffa a4506ceb bef9a3f7 c67178f2

Total: 64 constants


In [61]:
# Fix Problem 2
# Reference constants from FIPS PUB 180-4 Section 4.2.2
FIPS_CONSTANTS = [
    '428a2f98', '71374491', 'b5c0fbcf', 'e9b5dba5', '3956c25b', '59f111f1', '923f82a4', 'ab1c5ed5',
    'd807aa98', '12835b01', '243185be', '550c7dc3', '72be5d74', '80deb1fe', '9bdc06a7', 'c19bf174',
    'e49b69c1', 'efbe4786', '0fc19dc6', '240ca1cc', '2de92c6f', '4a7484aa', '5cb0a9dc', '76f988da',
    '983e5152', 'a831c66d', 'b00327c8', 'bf597fc7', 'c6e00bf3', 'd5a79147', '06ca6351', '14292967',
    '27b70a85', '2e1b2138', '4d2c6dfc', '53380d13', '650a7354', '766a0abb', '81c2c92e', '92722c85',
    'a2bfe8a1', 'a81a664b', 'c24b8b70', 'c76c51a3', 'd192e819', 'd6990624', 'f40e3585', '106aa070',
    '19a4c116', '1e376c08', '2748774c', '34b0bcb5', '391c0cb3', '4ed8aa4a', '5b9cca4f', '682e6ff3',
    '748f82ee', '78a5636f', '84c87814', '8cc70208', '90befffa', 'a4506ceb', 'bef9a3f7', 'c67178f2'
]


print("Comparing Generated vs Reference Constants:")

match_count = 0
for index, (generated, reference) in enumerate(zip(hex_constants, FIPS_CONSTANTS)):
    status = "PASS" if generated == reference else "FAIL"
    if generated == reference:
        match_count += 1
    print(f"{index:2d}. {status} {generated} | {reference}")

print(f"\nMatch Summary: {match_count}/64 constants correct")

if match_count == 64:
    print("\nALL 64 Passed")
    
    # Convert reference to numpy array for direct comparison
    reference_array = np.array([int(h, 16) for h in FIPS_CONSTANTS], dtype=np.uint32)
    
    if np.array_equal(K256, reference_array):
        print("\nArray comparison passed")
        print(f"Shape: {K256.shape}")
        print(f"Type: {K256.dtype}")
        print(f"Memory: {K256.nbytes} bytes")
        
        print("\nStatistical Properties")
        print(f"Minimum: 0x{K256.min():08x} (decimal: {K256.min()})")
        print(f"Maximum: 0x{K256.max():08x} (decimal: {K256.max()})")
        print(f"Mean:    0x{int(K256.mean()):08x}")
    else:
        print("Array comparison failed")
        
else:
    print(f"\nVALIDATION FAILED: {64 - match_count} mismatches")
    print("\nFirst few mismatches:")
    mismatch_count = 0
    for i, (gen, ref) in enumerate(zip(hex_constants, FIPS_CONSTANTS)):
        if gen != ref and mismatch_count < 5:
            print(f"  Index {i}: Generated {gen}, Expected {ref}")
            mismatch_count += 1


Comparing Generated vs Reference Constants:
 0. PASS 428a2f98 | 428a2f98
 1. PASS 71374491 | 71374491
 2. PASS b5c0fbcf | b5c0fbcf
 3. PASS e9b5dba5 | e9b5dba5
 4. PASS 3956c25b | 3956c25b
 5. PASS 59f111f1 | 59f111f1
 6. PASS 923f82a4 | 923f82a4
 7. PASS ab1c5ed5 | ab1c5ed5
 8. PASS d807aa98 | d807aa98
 9. PASS 12835b01 | 12835b01
10. PASS 243185be | 243185be
11. PASS 550c7dc3 | 550c7dc3
12. PASS 72be5d74 | 72be5d74
13. PASS 80deb1fe | 80deb1fe
14. PASS 9bdc06a7 | 9bdc06a7
15. PASS c19bf174 | c19bf174
16. PASS e49b69c1 | e49b69c1
17. PASS efbe4786 | efbe4786
18. PASS 0fc19dc6 | 0fc19dc6
19. PASS 240ca1cc | 240ca1cc
20. PASS 2de92c6f | 2de92c6f
21. PASS 4a7484aa | 4a7484aa
22. PASS 5cb0a9dc | 5cb0a9dc
23. PASS 76f988da | 76f988da
24. PASS 983e5152 | 983e5152
25. PASS a831c66d | a831c66d
26. PASS b00327c8 | b00327c8
27. PASS bf597fc7 | bf597fc7
28. PASS c6e00bf3 | c6e00bf3
29. PASS d5a79147 | d5a79147
30. PASS 06ca6351 | 06ca6351
31. PASS 14292967 | 14292967
32. PASS 27b70a85 | 27b70a85